# [SEMES] - 볼트 이상진단 - RegNet model

#### model 학습을 진행할 때 필요한 라이브러리 import

In [ ]:
import os


# 파이토치의 핵심 패키지(모델 구성 및 학습 등을 수행할 수 있는 기능을 제공)
import torch
# PyTorch에서 제공하는 신경망 모듈
import torch.nn as nn
# 학습에 사용되는 최적화 알고리즘
import torch.optim as optim
# PyTorch에서 이미지 데이터 처리와 관련된 함수와 모델들을 제공
import torchvision import models, datasets
# transforms 모듈은 데이터 전처리를 위한 함수들을 제공
import torchvision.transforms as transforms
# torchsummary는 PyTorch 모델의 요약 정보를 출력하는 라이브러리 / summary(model)로 요청
from torchsummary import summary
# DataLoader를 이용하여 데이터셋에서 미니배치(minibatch)를 추출 
from torch.utils.data import DataLoader
# 시간과 관련된 함수를 제공
import time

#### GPU 사용을 위한 설정

In [ ]:
# GPU가 사용 가능한 경우 cuda 사용 / GPU가 사용 불가능한 경우 CPU로 초기화하여 CPU 사용
if torch.cuda.is_available():
    device = torch.device("cuda:5")
    print("GPU is available. Using cuda")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU instead.")

#### 데이터 전처리

In [ ]:
# 학습데이터 전처리
train_transform = transforms.Compose([
    # 해상도를 (224,224)로 맞춰준다 (a fixed resolution of 224×224 is best, even at higher flops : [논문]Designing Network Design Spaces - [저자]Facebook AI Research (FAIR))
    transforms.Resize((224, 224)),
    # 이미지를 좌우로 뒤집어서 데이터 증강(augmentation)을 수행(확률을 높여준)
    transforms.RandomHorizontalFlip(),
    # 이미지를 PyTorch의 Tensor로 변환
    transforms.ToTensor(),
    # 흑백 이미지이기 때문에 1개의 채널을 정규화(흑백이미지는 보통 (평균 : 0.5 / 표준편차 : 0.5)로 정규화)
    transforms.Normalize([0.5], [0.5])
])

# 테스트데이터 전처리
test_transform = transforms.Compose([
    # 해상도를 (224,224)로 맞춰준다
    transforms.Resize((224, 224)),
    # 이미지를 PyTorch의 Tensor로 변환
    transforms.ToTensor(),
    # 흑백 이미지이기 때문에 1개의 채널을 정규화
    transforms.Normalize([0.5], [0.5])
])

# 데이터가 저장된 경로
data_dir = './semes_bolt'
print(os.path.join(data_dir, 'train'))

# 데이터가 저장된 경로에서 ImageFolder를 이용하여 이미지 데이터셋을 전처리한 후 로드(transforms_*==전처리 수행)
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), train_transform)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), test_transform)

# DataLoader를 이용 / 데이터셋에서 미니배치(minibatch)를 추출 
# (batch_size==미니배치의 크기 / shuffle==데이터셋을 섞을지 여부 / num_workers==데이터셋을 불러올 때 사용할 프로세스 수)
train_loader = DataLoader(train_datasets, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_datasets, batch_size=128, shuffle=True, num_workers=4)

# 수행 결과를 출력
print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

# 학습된 클래스 이름과 수행 결과를 출력
class_names = train_datasets.classes
print('클래스:', class_names)

#### RegNet_Y_1.6 모델 불러오기

In [ ]:
# RegNetY-16GF 모델 정의
model = torch.hub.load('facebookresearch/pytorchvideo', 'pytorchvideo_regnetx_16gf_kinetics400')

# 불러온 네트워크 모델의 출력 뉴런 수를 저장
num_features = model.fc.in_features
# 새로운 Fully Connected 레이어 추가
model.fc = nn.Linear(num_features, 3)

# GPU를 사용하기 위해 모델을 CUDA 디바이스로 보냄
model.to(device)

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)